In [8]:
import pandas as pd

data = pd.read_excel("T79JNAIS.xls")
processed = pd.DataFrame(columns=["Year","Month","Day","Birth"])
data

,T79JNAIS : Répartition quotidienne des naissances vivantes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,CHAMP : France métropolitaine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,JOUR,Total du mois,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29,30,31
3,Année de 1968 à 2018,Mois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968,Janvier,70409,2080.0,2319.0,2335.0,2338.0,2319.0,2212.0,2083.0,...,2268.0,2290.0,2345.0,2254.0,2324.0,2303.0,2232.0,2335,2352,2303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,NaN,Septembre,61834,1788.0,1761.0,1926.0,2175.0,2188.0,2147.0,2101.0,...,1940.0,1892.0,2145.0,2176.0,2229.0,2257.0,2219.0,1973,1875,so
613,NaN,Octobre,63588,2184.0,2238.0,2188.0,2208.0,2200.0,1912.0,1803.0,...,2023.0,2082.0,2174.0,1988.0,2155.0,1821.0,1893.0,2015,2155,2144
614,NaN,Novembre,59937,1849.0,1931.0,1796.0,1830.0,2173.0,2157.0,2143.0,...,2061.0,2098.0,1864.0,1743.0,2130.0,2113.0,2032.0,1983,2028,so
615,NaN,Décembre,59931,1838.0,1729.0,2006.0,2061.0,1997.0,2019.0,2049.0,...,1764.0,1592.0,1770.0,1551.0,1918.0,2193.0,2127.0,1719,1742,1881


In [16]:
years = []
months = []
days = []
births = []

year = -1
month = -1

for i in range(4,604,12):
    if not(data.iat[i,0]== "NaN") :
        year = data.iat[i,0]
        for j in range(0,12):
            month = data.iat[i+j,1]
            for k in range(0,31):
                if not data.iat[i+j,k+3]=="so":
                    years.append(year)
                    months.append(month)
                    days.append(k+1)
                    births.append(data.iat[i+j,k+3])
                    
    
        

In [17]:
years

['1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 '1968',
 

In [19]:
months

['Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Janvier',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Février',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 'Mars',
 

In [22]:
births


[2080.0,
 2319.0,
 2335.0,
 2338.0,
 2319.0,
 2212.0,
 2083.0,
 2252.0,
 2261.0,
 2295.0,
 2233.0,
 2284.0,
 2168.0,
 2166.0,
 2366.0,
 2368.0,
 2331.0,
 2267.0,
 2286.0,
 2301.0,
 2139.0,
 2268.0,
 2290.0,
 2345.0,
 2254.0,
 2324.0,
 2303.0,
 2232.0,
 2335,
 2352,
 2303,
 2327.0,
 2399.0,
 2228.0,
 2198.0,
 2391.0,
 2516.0,
 2398.0,
 2402.0,
 2515.0,
 2410.0,
 2189.0,
 2268.0,
 2359.0,
 2396.0,
 2392.0,
 2298.0,
 2204.0,
 2136.0,
 2299.0,
 2372.0,
 2290.0,
 2247.0,
 2286.0,
 2342.0,
 2081.0,
 2296.0,
 2272.0,
 2261.0,
 2305,
 2371.0,
 2385.0,
 2200.0,
 2394.0,
 2391.0,
 2416.0,
 2386.0,
 2352.0,
 2454.0,
 2235.0,
 2470.0,
 2389.0,
 2269.0,
 2399.0,
 2409.0,
 2477.0,
 2274.0,
 2391.0,
 2450.0,
 2442.0,
 2473.0,
 2355.0,
 2370.0,
 2174.0,
 2501.0,
 2441.0,
 2294.0,
 2356.0,
 2441,
 2456,
 2288,
 2448.0,
 2361.0,
 2381.0,
 2370.0,
 2344.0,
 2454.0,
 2208.0,
 2344.0,
 2348.0,
 2344.0,
 2434.0,
 2414.0,
 2435.0,
 2275.0,
 2285.0,
 2516.0,
 2464.0,
 2399.0,
 2431.0,
 2502.0,
 2391.0,
 2518.

In [26]:
d = {'year':years,'month':months,'day':days,'birth':births}
processed = pd.DataFrame(d)

In [27]:
processed

,year,month,day,birth
0,1968,Janvier,1,2080.0
1,1968,Janvier,2,2319.0
2,1968,Janvier,3,2335.0
3,1968,Janvier,4,2338.0
4,1968,Janvier,5,2319.0
...,...,...,...,...
18258,2017,Décembre,27,2238.0
18259,2017,Décembre,28,2212.0
18260,2017,Décembre,29,2213.0
18261,2017,Décembre,30,1788.0


In [35]:
period = 29.53
reference = 0
moon = 0
moonTotal = 0
for i in range(processed.shape[0]):
    reference += processed.at[i,'birth']
    if i == round(period * moonTotal,0) + 14:
        moonTotal += 1
        moon += processed.at[i,'birth']


In [37]:
ReferenceMean = reference/18263
MoonMean = moon/moonTotal
Ratio = MoonMean/ReferenceMean
Ratio

1.0043471832547397

In [38]:
ReferenceMean

2121.3138586212563

In [39]:
MoonMean

2130.535598705502